In [69]:
## MAIN function

def dot_bracket(seq, bp_list):
    ''' dot_bracket takes a list of base pairs and converts to dot bracket notation
    
    args:
        seq: sequence of interest as a string
        bp_list: of list of tuples where the tuples are the indeces of the bp in increasing order (bp[0]<bp[1])
        
    output:
        prints the sequence and the dot bracket notation
        
    returns:
        returns a string with the dot bracket notation
    '''
    dot_bracket = "."*len(seq)
    
    if not is_PK(bp_list):
        for bp in bp_list:
            dot_bracket = dot_bracket[:bp[0]]+"("+dot_bracket[bp[0]+1:bp[1]]+ ")"+dot_bracket[bp[1]+1:]

    else:
        brackets = [("(",")"),("[","]"),("{","}"),("<",">")]
        groups = group_into_non_conflicting_bp(is_PK_and_get_list_conflicts(bp_list))
        if len(groups) > len(brackets):
            print("WARNING: PK too complex")
        for group,bracket in zip(groups,brackets):
            for bp in group:
                dot_bracket = dot_bracket[:bp[0]]+bracket[0]+dot_bracket[bp[0]+1:bp[1]]+bracket[1]+dot_bracket[bp[1]+1:]
    
    print(seq)
    print(dot_bracket)
    return dot_bracket  

In [70]:
## HELPER functions

def is_PK(bp_list):
    '''checks if a given bp_list represents a PK
    Args:
        bp_list: of list of tuples where the tuples are the indeces of the bp in increasing order (bp[0]<bp[1])
    
    returns:
        True if it is a psuedoknot
    '''
    if bp_list == []:
        return False
    else:
        current_bp = bp_list[0]
        for bp in bp_list[1:]:
            if ((current_bp[0] < bp[0] and bp[0] < current_bp[1] and current_bp[1] < bp[1])
                or (current_bp[0] > bp[0] and current_bp[0] < bp[1] and bp[1] < current_bp[1])):
                return True
        return is_PK(bp_list[1:])

def is_PK_and_get_list_conflicts(bp_list):
    '''given a bp_list gives the list of conflicts bp-s which indicate PK structure
    Args:
        bp_list: of list of tuples where the tuples are the indeces of the bp in increasing order (bp[0]<bp[1])
    
    returns:
        List of conflicting basepairs, where conflicting is pairs of base pairs that are intertwined.
    '''    if len(bp_list) <= 1:
        return []
    else:
        current_bp = bp_list[0]
        conflicts = []
        for bp in bp_list[1:]:
            if ((current_bp[0] < bp[0] and bp[0] < current_bp[1] and current_bp[1] < bp[1])
                or (current_bp[0] > bp[0] and current_bp[0] < bp[1] and bp[1] < current_bp[1])):
                conflicts.append((current_bp,bp))
        return conflicts + is_PK_and_get_list_conflicts(bp_list[1:])
    
def group_into_non_conflicting_bp(conflict_list):
    ''' given a conflict list from is_PK_and_get_list_conflicts, group basepairs into groups that do not conflict
    
    Args
        conflict_list: list of pairs of base_pairs that are intertwined basepairs
        
    Returns:
        groups of baspairs that are not intertwined
    '''
    non_redudant_bp_list = get_non_redudant_bp_list(conflict_list)
    groups = []
    while non_redudant_bp_list != []:
        current_bp = non_redudant_bp_list[0]
        current_bp_conflicts = []
        for conflict in conflict_list:
            if current_bp == conflict[0]:
                current_bp_conflicts.append(conflict[1])
            elif current_bp == conflict[1]:
                current_bp_conflicts.append(conflict[0])
        group = [bp for bp in non_redudant_bp_list if bp not in current_bp_conflicts]
        groups.append(group)
        non_redudant_bp_list = current_bp_conflicts
        conflict_list = [conflict for conflict in conflict_list if conflict[0] not in group and conflict[1] not in group]
    return groups
    
def get_non_redudant_bp_list(conflict_list):
    ''' given a conflict list get the list of nonredundant basepairs this list has
    
    Args:
        conflict_list: list of pairs of base_pairs that are intertwined basepairs
    returns:
        list of basepairs in conflict list without repeats

    '''
    non_redudant_bp_list = []
    for conflict in conflict_list:
        if conflict[0] not in non_redudant_bp_list:
            non_redudant_bp_list.append(conflict[0])
        if conflict[1] not in non_redudant_bp_list:
            non_redudant_bp_list.append(conflict[1])
    return non_redudant_bp_list


In [71]:
SL = [(0,10),(1,9),(2,8),(3,7)]
SL2 = [(0,5),(1,4),(6,10)]
PK = [(0,6),(1,5),(2,9),(3,8)]

In [72]:
print(is_PK(SL))
print(is_PK(SL2))
print(is_PK(PK))

False
False
True


In [73]:
print(is_PK_and_get_list_conflicts(PK))

[((0, 6), (2, 9)), ((0, 6), (3, 8)), ((1, 5), (2, 9)), ((1, 5), (3, 8))]


In [74]:
dot_bracket("AAAAAAAAAAA",SL)
dot_bracket("AAAAAAAAAAA",SL2)
dot_bracket("AAAAAAAAAAA",PK)


AAAAAAAAAAA
((((...))))
AAAAAAAAAAA
((..))(...)
AAAAAAAAAAA
(([[.)).]].


'(([[.)).]].'